# Videos Pipeline

In [16]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# load helper functions
%run -i "0. Functions_Clases Pipeline.py"
%run -i "Line.py"

# Load Camera calibration params
[ret, mtx, dist, rvecs, tvecs] = pickle.load(open( "pickle_data/camera_calibration_params.p", "rb" ) )


[NbConvertApp] Converting notebook 0. Functions_Clases Pipeline.ipynb to script
[NbConvertApp] Writing 20623 bytes to 0. Functions_Clases Pipeline.py


In [17]:
def process_frame(img):
    
    ###### Resize image
    img = resizeImage(img)

    ###### Undistort image
    undistorted = undistort_image(img, mtx, dist)
  
    ###### Color Enhancement

    imageCE = colorEnhancement(img)
   
    ###### GrayScale
    imageGray = grayscale(imageCE)


    ###### Gauss Smoothing
    imageGauss = gaussian_blur(imageGray,kernel_size=5) 
    
    #### Edge detection
    sbinary = sobel_thresh(imageGauss, sobel_kernel=5, x_thresh=[80,100], y_thresh=[40,100], 
                           mag_thresh=[50,255], dir_thresh=[100,200])

    #### ROI
    ysize =sbinary.shape[0]
    xsize =sbinary.shape[1]

    ROI_upperWidth = 280 #Width of the upper horizontal straight in px
    ROI_upperHeight = 260 #Height of the upper horizontal straight from the bottom of the image in px
    ROI_lowerWidth = 950 #Width of the lower horizontal straight in px
    ROI_lowerHeight = 50  #Height of the lower horizontal straight  from the bottom of the image in px      

    limitLL = ((xsize/2)-(ROI_lowerWidth/2),ysize-ROI_lowerHeight);
    limitLR = (xsize - ((xsize/2)-(ROI_lowerWidth/2)),ysize-ROI_lowerHeight);
    limitUL = ((xsize/2)-(ROI_upperWidth/2), ysize-ROI_upperHeight);
    limitUR = ((xsize/2)+(ROI_upperWidth/2), ysize-ROI_upperHeight);
    vertices = np.array([[limitLL,limitUL,limitUR , limitLR]], dtype=np.int32)
    
    imageROI = region_of_interest(sbinary,vertices,)

    
    #### Perspective transform
    warped_img,M, Minv = warp_image(imageROI, hwidth = 250 ,offset = 0, height = -800, )
    
    #### Find lines
    
    # Find x line poitns based on histogram values
    leftx_base, rightx_base  = find_lane_x_points(warped_img)
    
    # Update x base points
    lineLeft.updateXbase(leftx_base)
    lineRight.updateXbase(rightx_base)
    
    
    ## Find pixels
    ### Left line
    if lineLeft.missdetections > 0 or np.any((lineLeft.recent_poly_fits == 0)):
        # Find lane pixels
        leftx, lefty, rightx, righty, out_img = find_lane_pixels(warped_img, lineLeft.bestx, 
                                                                 lineRight.bestx, showRectangles = False)
        
        # Update lane pixels
        lineLeft.updatePixels(leftx, lefty)   

        # Search blindly image
        coeffs_fit_L, lineDetectedL, left_fitx, ploty, img_line  = fit_polynomial(out_img, lineLeft.allx,
                                                                                  lineLeft.ally, drawPoly = False)
    ## Speed up pixel search with coef with area search    
    else:
        # Search based on coefs
        leftx, lefty, coeffs_fit_L, lineDetectedL, left_fitx, out_img = search_around_poly(warped_img, lineLeft)
        lineLeft.updatePixels(leftx, lefty)  

    #Right line
    if lineRight.missdetections > 0 or np.any((lineRight.recent_poly_fits == 0)):
        ## Update lane pixels
        if not("rightx" in locals()) or not("righty" in locals()) :
            leftx, lefty, rightx, righty, out_img = find_lane_pixels(warped_img, lineLeft.bestx, 
                                                                     lineRight.bestx, showRectangles = False)
        # Update lane pixels
        lineRight.updatePixels(rightx, righty)
        
        # Search blindly image
        coeffs_fit_R, lineDetectedR, right_fitx, ploty, img_line  = fit_polynomial(out_img, lineRight.allx,
                                                                                   lineRight.ally, drawPoly = False)
    else:
        # Search based on coefs
        rightx, righty, coeffs_fit_R, lineDetectedR, right_fitx, out_img = search_around_poly(out_img, lineRight)   
        lineRight.updatePixels(rightx, righty)
        
           
    #Update line class instances
    lineLeft.updateCoeffsLine(lineDetectedL, coeffs_fit_L, left_fitx, 
                              lineLeft.poly_ploty,coefLimits=[0.1,1,200],movingAvg=5)
    lineRight.updateCoeffsLine(lineDetectedR, coeffs_fit_R,right_fitx,
                               lineRight.poly_ploty,coefLimits=[0.1,1,200],movingAvg=5)
        

        
    #Sanity check line width
    if lineLeft.detected and lineRight.detected:
        if left_fitx.any and right_fitx.any and len(lineLeft.recent_poly_fits) >1 and len(lineRight.recent_poly_fits) >1:  # If vectors are empty do nothing
            # If separation ok, update stage vectors
            if rightSeparation(left_fitx, right_fitx, limitDist = 100):
                lineLeft.poly_best_fit_staged = lineLeft.poly_best_fit
                lineRight.poly_best_fit_staged = lineRight.poly_best_fit
                #lineLeft.poly_plotx_staged = left_fitx 
                #lineRight.poly_plotx_staged = right_fitx
                pass
            else: # If separation not ok, use stage vectors
                print("Wrong separation between lines")
                ### Left line
                #Discard last fit
                lineLeft.recent_poly_fits = lineLeft.recent_poly_fits[:-1]
                
                # Reuse best fit if exists
                
             
                if  len(lineLeft.poly_best_fit_staged) > 2 :
                    lineLeft.poly_best_fit = lineLeft.poly_best_fit_staged
                    #print("1")
                    #print(lineLeft.poly_best_fit_staged)
                else:         
                    lineLeft.poly_best_fit = lineLeft.recent_poly_fits[-1]
                    #print("2")
                    #print(lineLeft.poly_best_fit)
                    
                ### Right line
                #Discard last fit
                lineRight.recent_poly_fits = lineRight.recent_poly_fits[:-1]
                
                # Reuse best fit if exists
                if  len(lineRight.poly_best_fit_staged) > 2 :
                    lineRight.poly_best_fit = lineRight.poly_best_fit_staged
                else:
                    lineRight.poly_best_fit = lineRight.recent_poly_fits[-1]
                    
                
                # Compute new plot
                lineLeft.poly_plotx = np.polyval(lineLeft.poly_best_fit, lineLeft.poly_ploty)
                lineRight.poly_plotx = np.polyval(lineRight.poly_best_fit, lineRight.poly_ploty)
                # Increase missdetections
                lineLeft.missdetections += 1
                lineRight.missdetections += 1
            
    ##Do sanity checks on lines
    lineLeft.sanityCheck(limit=10)
    lineRight.sanityCheck(limit=10)
    
    ### Unwarp images
    
    #color_warp = np.zeros_like(out_img).astype(np.uint8)
    color_warp = out_img
    #color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    pts_left = np.array([np.transpose(np.vstack([lineLeft.poly_plotx, lineLeft.poly_ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([lineRight.poly_plotx, lineRight.poly_ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
     
    #Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    
    newwarp = cv2.warpPerspective(color_warp, M, (img.shape[1], img.shape[0])) 
    result = cv2.addWeighted(img, 1, newwarp, 0.5, 0)
    
    
    ### Anotate Radius of curvature   
    
    diff, mean, text = checkRadius(lineLeft, lineRight,limitStraight=10000, movingAvg = 1 )

    result_annotated = cv2.putText(result, text, org= (50, 50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                   fontScale=2, color= (255, 255, 255), thickness=2, lineType=cv2.LINE_AA)
    

    ### Anotate Vehicle position
    dev, text = calculateDeviation(result, lineLeft,lineRight,)

    result_annotated = cv2.putText(result, text, org= (50, 100), fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                       fontScale=2, color= (255, 255, 255), thickness=2, lineType=cv2.LINE_AA)
    
    # Anotate missdetections
    #result_annotated = cv2.putText(result, str(lineLeft.missdetections),  org= (50, 200), fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
     #                fontScale=1, color= (255, 255, 255), thickness=2, lineType=cv2.LINE_AA)
    
    #result_annotated = cv2.putText(result, str(lineRight.missdetections),  org= (50, 300), fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
#                       fontScale=1, color= (255, 255, 255), thickness=2, lineType=cv2.LINE_AA)
      
 
    #out = np.dstack((out_img*255, out_img*255, out_img*255))
    
    # Show ROI
    #cv2.polylines(result,[vertices],True,(0,0,255),thickness=5)
    
    return result_annotated

## Video 1

In [18]:
# Instanciate cLine classes
lineLeft = Line()
lineRight = Line()


white_output = 'output_videos/project_video.mp4'
clip1 = VideoFileClip("test_videos/project_video.mp4")
#clip1 = VideoFileClip("test_videos/project_video.mp4").subclip(30,40)
white_clip = clip1.fl_image(process_frame) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   0%|                                                            | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_videos/project_video.mp4.
Moviepy - Writing video output_videos/project_video.mp4



t:  83%|████████████████████████████████████████▌        | 1042/1260 [04:19<00:52,  4.13it/s, now=None]

missdetection
[ 2.00591734e-03 -1.83909703e+00  4.24037828e+02]


t:  83%|████████████████████████████████████████▊        | 1050/1260 [04:21<00:49,  4.24it/s, now=None]

Discarded X point


t:  83%|████████████████████████████████████████▊        | 1051/1260 [04:21<00:50,  4.16it/s, now=None]

Discarded X point


Moviepy - Done !
Moviepy - video ready output_videos/project_video.mp4
Wall time: 5min 15s


In [19]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Video 2

In [5]:
# Instanciate cLine classes
lineLeft = Line()
lineRight = Line()


challenge_output = 'output_videos/challenge_video.mp4'
clip2 = VideoFileClip("test_videos/challenge_video.mp4")

challenge_clip = clip2.fl_image(process_frame) #NOTE: this function expects color images!!
%time challenge_clip.write_videofile(challenge_output, audio=False)

t:   0%|                                                             | 0/485 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_videos/challenge_video.mp4.
Moviepy - Writing video output_videos/challenge_video.mp4

missdetection
[-1.92871935e-03  2.40260878e+00 -7.44370229e+02]


t:   1%|▎                                                    | 3/485 [00:02<05:19,  1.51it/s, now=None]

Wrong separation between lines


t:   1%|▍                                                    | 4/485 [00:02<03:59,  2.01it/s, now=None]

...

Wrong separation between lines


t: 100%|███████████████████████████████████████████████████| 485/485 [02:00<00:00,  4.22it/s, now=None]

Wrong separation between lines


Wrong separation between lines


Moviepy - Done !
Moviepy - video ready output_videos/challenge_video.mp4
Wall time: 2min 1s


In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

## Video 3

In [7]:
# Instanciate cLine classes
lineLeft = Line()
lineRight = Line()


harder_challenge_output = 'output_videos/harder_challenge_video.mp4'
clip3 = VideoFileClip("test_videos/harder_challenge_video.mp4")

harder_challenge_clip = clip3.fl_image(process_frame) #NOTE: this function expects color images!!
%time harder_challenge_clip.write_videofile(harder_challenge_output, audio=False)

t:   0%|                                                            | 0/1199 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_videos/harder_challenge_video.mp4.
Moviepy - Writing video output_videos/harder_challenge_video.mp4



t:   6%|██▊                                                | 66/1199 [00:18<04:58,  3.79it/s, now=None]

Wrong separation between lines


t:   6%|██▊                                                | 67/1199 [00:18<04:58,  3.79it/s, now=None]

...

t: 100%|████████████████████████████████████████████████▉| 1197/1199 [05:34<00:00,  3.51it/s, now=None]

Wrong separation between lines


t: 100%|█████████████████████████████████████████████████| 1199/1199 [05:35<00:00,  3.67it/s, now=None]

Wrong separation between lines


Moviepy - Done !
Moviepy - video ready output_videos/harder_challenge_video.mp4
Wall time: 5min 37s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(harder_challenge_output))

## Helper code

In [9]:
#for i in range(1,10):
#    clip2.save_frame("test_video_frames/challenge_{}s.png".format(i), t=i) # saves the frame a t=xs